In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

# FOR PHASE 1 of Training
def lr_schedule_1(epoch): 
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

# FOR PHASE 2 of Training
def lr_schedule_2(epoch):  
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    elif epoch > 45:
        lrate = 0.0003        
    return lrate

# FOR PHASE 3 of Training
def lr_schedule_3(epoch): 
    lrate = 0.001
    #if epoch > 15:
    #    lrate = 0.0005
    #elif epoch > 25:
    #    lrate = 0.0003        
    return lrate
    
#Data normalization as per CIFAR-10 dataset instructions - https://www.cs.toronto.edu/~kriz/cifar.html
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)


170500096/170498071 [==============================] - 10s 0us/step


In [ ]:
# Construct CNN

weight_decay = 1e-4
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# Add softmax (output) layer
model.add(Dense(num_classes, activation='softmax'))

#data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Only one model.fit call of the 3 listed below must be uncommented while execution

# For Phase 1: 125 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_1)])

# For Phase 2: 60 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=60,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_2)])

# For Phase 3: 30 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=30,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])


In [ ]:
y_pred_test = model.predict(x_test)
#y_pred_test
joint_prob = np.zeros((10,10)) #for CIFAR 10 (i,j) - state (true label) i, action(prediction prob) j
for i in range(10):
  idx  = np.where(np.argmax(y_test,1)==i)
  for j in range(10):
    joint_prob[i,j] = np.mean(y_pred_test[idx,j])

joint_prob = joint_prob*0.1 # prior pmf over the set of true image classes in CIFAR-10  is uniform.

# joint_prob[x,a] is the joint probability that the true image class is 'x' and predicted image class is 'a'
joint_prob

array([[7.71876514e-02, 1.96646098e-03, 9.53053460e-03, 1.15394173e-03,
        1.24988602e-03, 4.45861230e-04, 1.77954677e-03, 1.62081420e-03,
        3.12381871e-03, 1.94148254e-03],
       [5.79802599e-04, 9.44558620e-02, 3.91505705e-04, 1.40780467e-04,
        3.97567695e-05, 1.46135746e-04, 1.00040911e-03, 1.71544461e-04,
        4.11856454e-04, 2.66234707e-03],
       [2.73436513e-03, 2.39159539e-04, 8.06261122e-02, 1.27323633e-03,
        2.02548448e-03, 1.48051493e-03, 9.23073143e-03, 1.64296273e-03,
        3.62588931e-04, 3.84852639e-04],
       [1.92269254e-03, 8.45141988e-04, 1.32170662e-02, 4.35196102e-02,
        3.44821066e-03, 1.07349418e-02, 2.00739935e-02, 4.07545865e-03,
        9.03555099e-04, 1.25932945e-03],
       [1.06691681e-03, 1.94567873e-04, 1.14292018e-02, 2.39615366e-03,
        6.49595201e-02, 1.66074056e-03, 1.34692594e-02, 4.42119874e-03,
        2.13169376e-04, 1.89271849e-04],
       [8.79888143e-04, 4.23107855e-04, 1.07444644e-02, 8.00894648e-03,
   

In [ ]:
#Code structure credits :  https://github.com/abhijeet3922/Object-recognition-CIFAR-10/blob/master/cifar10_90.py